In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time 
import pandas as pd
import pymongo
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Document, Text, Integer
from elasticsearch.helpers import bulk
import numpy as np

In [3]:
df = pd.read_csv('data_youtube.csv',delimiter=',')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1477 entries, 0 to 1476
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Jeu                       1477 non-null   object
 1   Titre                     1477 non-null   object
 2   Chaîne                    1477 non-null   object
 3   Nombre de vues            1477 non-null   object
 4   Date de la mise en ligne  1477 non-null   object
dtypes: object(5)
memory usage: 57.8+ KB


In [57]:
documents = df.fillna("").to_dict(orient="records")